<a href="https://colab.research.google.com/github/Morilla123/Curso/blob/main/Copia_de_Transformers_medicina_extraccion_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraccion de informacion de reportes medicos

El objetivo de este proyecto es extraer desde un reporte medico que contiene texto libre, informacion del paciente, informacion de la medicacion del paciente, y una imagen de un radiografia de torax, lo siguiente:

- Texto del reporte de-identificado (sin informacion personal del paciente)
- Medicamentos con los que es tratado el paciente
- Prediccion de si en la radiografia es normal o presenta neumonia

Documento a analizar: [Reporte](https://drive.google.com/file/d/15zA5XDQfrVIYZk0gHGLxQqt_up7-NjuP/view?usp=sharing)

*1*) De-identificar texto (quitar informacion personal)

2) Obtener medicamentos del paciente

3) Predecir si la radiografia corresponde a un paciente normal o con neumonia

## Test ##


Import librerias


In [ ]:
import requests

In [ ]:
def Use_API(text, url="https://bern.korea.ac.kr/plain"):          # API = https://bern.korea.ac.kr/#
  return requests.post(url, data={'sample_text': text}).json()

In [ ]:
def Extraxtion_data(text,out,type):

  denotations=out['denotations']
  relavent_terms=[]
  
  if type=='drug':
    drug_terms=[]
    for i in denotations:
      if i['obj']== 'drug':
        drug_terms.append((i['span']['begin'],i['span']['end']))
   
    for i in drug_terms:
      start=i[0]
      end=i[1]
      relavent_terms.append(text[start:end])

  if type=='disease':
    species_terms=[]
    for i in denotations:
      if i['obj']== 'disease':
        species_terms.append((i['span']['begin'],i['span']['end']))
    
    for i in species_terms:
      start=i[0]
      end=i[1]
      relavent_terms.append(text[start:end])


  # if type=='species':
  #   species_terms=[]
  #   for i in denotations:
  #     if i['obj']== 'species':
  #       species_terms.append((i['span']['begin'],i['span']['end']))
    
  #   for i in species_terms:
  #     start=i[0]
  #     end=i[1]
  #     relavent_terms.append(text[start:end])

  return(relavent_terms)

In [10]:
# text = "X-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. She was initially treated with ice, an ace wrap, crutches and mild pain medications (Tylenol with codeine)"
text = "Name: Matias Molinas Birth date: 10/03/1918 (54) 342- 5678906 Significant Medical History Mr. Matias Molinas is a 52 y/o accountant with hypercholesterolemia and polycythemia vera who has been in relatively good health. Diagnostic: Patient needs to take 50 mg of ibuprofen. Chest X-Ray:"

In [ ]:
out=Use_API(text)
# print(out)
# print(Extraxtion_data(text,out,'drug'))
List_Drugs = Extraxtion_data(text,out,'drug')
# print(Extraxtion_data(text,out,'disease'))
List_Disease = Extraxtion_data(text,out,'disease')

['ibuprofen']
['hypercholesterolemia', 'polycythemia vera']


In [ ]:
print(List_Drugs)

['ibuprofen']


## image extract ##

In [1]:
!pip install PyMuPDF

     |████████████████████████████████| 8.7 MB 3.5 MB/s 


In [2]:
import fitz

In [3]:
file_path = input("Enter the PDF file path")
pdf_file = fitz.open(file_path)

#Reading the location where to save the file
location = input("Enter the location to save: ")

#finding number of pages in the pdf
number_of_pages = len(pdf_file)

#iterating through each page in the pdf
for current_page_index in range(number_of_pages):
  #iterating through each image in every page of PDF
  for img_index,img in enumerate(pdf_file.getPageImageList(current_page_index)):
        xref = img[0]
        image = fitz.Pixmap(pdf_file, xref)
        #if it is a is GRAY or RGB image
        if image.n < 5:        
            image.writePNG("{}/image{}-{}.png".format(location,current_page_index, img_index))
        #if it is CMYK: convert to RGB first
        else:                
            new_image = fitz.Pixmap(fitz.csRGB, image)
            new_image.writePNG("{}/image{}-{}.png".foramt(location,current_page_index, img_index))
            

Enter the PDF file pathReport.pdf
Enter the location to save: /content/pdf


Deprecation: 'getPageImageList' removed from class 'Document' after v1.19.0 - use 'get_page_images'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19.0 - use 'save'.


## Anonimizacion de nombres

In [8]:
%%capture
! pip install transformers datasets
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf
!pip install spacy
from spacy import displacy
import spacy

text = "Name: Matias Molinas Birth date: 10/03/1918 (54) 342- 5678906 Significant Medical History Mr. Matias Molinas is a 52 y/o accountant with hypercholesterolemia and polycythemia vera who has been in relatively good health. Diagnostic: Patient needs to take 50 mg of ibuprofen. Chest X-Ray:"


In [9]:
nlp = spacy.load("en_core_web_sm")

In [14]:
doc = nlp(text)
displacy.render(doc, style='ent')

'<div class="entities" style="line-height: 2.5; direction: ltr">Name: \n<mark class="entity" style="background: #aa9cfc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Matias Molinas Birth\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">PERSON</span>\n</mark>\n date: \n<mark class="entity" style="background: #bfe1d9; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    10/03/1918\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">DATE</span>\n</mark>\n (\n<mark class="entity" style="background: #e4e7d2; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    54\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; 

In [15]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Matias Molinas Birth PERSON
10/03/1918 DATE
54 CARDINAL
5678906 CARDINAL
Matias Molinas PERSON
52 y QUANTITY
50 mg TIME
Chest X-Ray PERSON


In [19]:
# The start_char of each entity PER
per_starts = [ent.start_char for ent in doc.ents if ent.label_ == 'PERSON']

# The end_char of each entity PER
per_ends = [ent.end_char for ent in doc.ents if ent.label_ == 'PERSON']

print(f'per_start: {per_starts}\nper_ends: {per_ends}')

per_start: [6, 94, 274]
per_ends: [26, 108, 285]


In [20]:
# colaboration https://github.com/mnsosa/ner/blob/main/ner_spanish_ialab.ipynb
# Anonymization
text_anon = doc.text[0:per_starts[0]]
text_anon = text_anon + '<NOMBRE>'

per = 1
while per < len(per_starts):
    text_anon = text_anon + doc.text[per_ends[per-1]:per_starts[per]] + '<NOMBRE> '
    per = per + 1

In [21]:
text_anon

'Name: <NOMBRE> date: 10/03/1918 (54) 342- 5678906 Significant Medical History Mr. <NOMBRE>  is a 52 y/o accountant with hypercholesterolemia and polycythemia vera who has been in relatively good health. Diagnostic: Patient needs to take 50 mg of ibuprofen. <NOMBRE> '

## VisionTreansformers

In [1]:
%%capture
!pip install kaggle --upgrade

In [ ]:
! mkdir ~/.kaggle # simplemente creo una carpeta en colab pero en la raiz

In [6]:
! cp kaggle.json ~/.kaggle/ # mueve el kaggle jonson para la carpeta que acabamos de crear

In [7]:
! chmod 600 ~/.kaggle/kaggle.json # es como el permiso para que funque

In [10]:
! kaggle datasets download paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:26<00:00, 32.8MB/s]
100% 2.29G/2.29G [00:26<00:00, 91.3MB/s]


In [ ]:
! unzip chest-xray-pneumonia.zip

In [12]:
%%capture
!pip install -U tensorflow-addons

In [13]:
%%capture
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa